In [ ]:
import yfinance as yf
import logging
from datetime import datetime

# Configure logging
logging.basicConfig(filename='trading_bot.log', level=logging.INFO,
                    format='%(asctime)s %(message)s')

# Base class for trading strategies
class TradingStrategy:
    def __init__(self, name):
        self.name = name

    def generate_signal(self, price_data):
        raise NotImplementedError("Subclasses should implement this method")

# Simple Moving Average Strategy
class SMAStrategy(TradingStrategy):
    def __init__(self, short_window, long_window):
        super().__init__("Simple Moving Average Strategy")
        self.short_window = short_window
        self.long_window = long_window

    def generate_signal(self, price_data):
        if len(price_data) < self.long_window:
            return "hold"
        
        short_avg = sum(price_data[-self.short_window:]) / self.short_window
        long_avg = sum(price_data[-self.long_window:]) / self.long_window
        current_price = price_data[-1]
        
        logging.info(f"Short Avg: {short_avg}, Long Avg: {long_avg}, Current Price: {current_price}")
        
        if short_avg > long_avg:
            return "buy"
        elif short_avg < long_avg:
            return "sell"
        else:
            return "hold"

# Trade management class
class Trade:
    def __init__(self, strategy_name, signal, amount):
        self.strategy_name = strategy_name
        self.signal = signal
        self.amount = amount
        self.timestamp = datetime.now()

    def execute(self):
        logging.info(f"Executing {self.signal} for {self.amount} units using {self.strategy_name} at {self.timestamp}")

# Mock trading API
class MockTradingAPI:
    def __init__(self, balance):
        self.balance = balance

    def place_order(self, trade, price):
        if trade.signal == "buy" and self.balance >= trade.amount * price:
            self.balance -= trade.amount * price
            logging.info(f"Placed buy order for {trade.amount} units at {price}.")
        elif trade.signal == "sell":
            self.balance += trade.amount * price
            logging.info(f"Placed sell order for {trade.amount} units at {price}.")
        else:
            logging.info("Insufficient balance or invalid signal.")

    def get_balance(self):
        return self.balance

# Main trading system
class TradingSystem:
    def __init__(self, api, strategy, symbol):
        self.api = api
        self.strategy = strategy
        self.symbol = symbol
        self.price_data = []

    def fetch_price_data(self):
        data = yf.download(tickers=self.symbol, period='1d', interval='1m')
        if not data.empty:
            price = data['Close'].iloc[-1]  # closing price of the last minute
            self.price_data.append(price)
            if len(self.price_data) > self.strategy.long_window:
                self.price_data.pop(0)
            logging.info(f"Fetched new price data: {price}")
        else:
            logging.info("No data fetched")

    def run(self):
        self.fetch_price_data()
        signal = self.strategy.generate_signal(self.price_data)
        logging.info(f"Generated signal: {signal}")
        if signal in ["buy", "sell"]:
            trade = Trade(self.strategy.name, signal, 1)  # Assuming a fixed trade amount of 1 unit
            trade.execute()
            self.api.place_order(trade, self.price_data[-1])

# Example usage
if __name__ == "__main__":
    symbol = 'AAPL'

    api = MockTradingAPI(balance=10000)
    strategy = SMAStrategy(short_window=3, long_window=5)
    system = TradingSystem(api, strategy, symbol)

    # Simulating a trading session with 10 iterations
    for _ in range(10):
        system.run()
        print(f"Remaining balance: {api.get_balance()}")
